In [206]:
import ipyvuetify as v
import ipywidgets as widgets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from IPython.display import display, clear_output

### SQLytes Inventory Tracking



In [207]:
# URL for localhost backend when using Docker.
backend = 'http://127.0.0.1:5000/sqlytes/'
# warehouse endpoint
endpoint = '/warehouse'

In [208]:
def getAvailableWarehouses():
    """Return all IDs of the warehouses in the Data Base."""
    response = requests.get(f'{backend}{"warehouse"}')
    # List to store all available wid's.
    wids = []
    # Check if response was succesful.
    if response.status_code == 200:
        # Parse JSON Data from response.
        data = response.json()
        for record in data['Warehouses']:
            wids.append(record['wid'])
    else:
        print('Error {}: Could not get data from the backend.'.format(response.status_code))
    return wids

In [209]:
available_warehouses = getAvailableWarehouses()

In [210]:
# print("Available Warehouse IDs: ",getAvailableWarehouses())

### Choose a Warehouse ID to get all Parts from.

In [211]:
selection = widgets.Dropdown(
    options = available_warehouses,
    value = available_warehouses[0],
    description="wid"
)

Selected Warehouse: 3


Selected Warehouse: 6


Selected Warehouse: 8


Selected Warehouse: 11


In [212]:
output_area = widgets.Output()

In [213]:
def getWarehousePartsAPI(selected_warehouse):
    """Obtain all Parts for a warehouse.

    Args:
        wid (int): ID of the warehouse of which the parts are going to be showed.
        endpoint (str): Endpoint to be connected to the backend route to obtain the data.
    """
    endpoint = f'warehouse/{selected_warehouse}/parts'
    response = requests.get(f'{backend}{endpoint}')
    with output_area:
        clear_output(wait=True)  # Clear the previous output
        if response.status_code == 200:
            data = response.json()
            all_parts_df = pd.DataFrame(data["WarehouseParts"])
            display(all_parts_df)
        elif response.status_code == 404:
            print('Error {}: Warehouse has no parts'.format(response.status_code))
        else:
            print(f'{backend}{endpoint}')
            print('Error {}: Could not get data from the backend.'.format(response.status_code))

In [214]:
# getWarehousePartsAPI(selection.value)

In [215]:
def on_selection_change(change):
    if isinstance(change, int):
        # Directly received the selected value, not an event dictionary
        selected_warehouse = change
    elif change['type'] == 'change' and change['name'] == 'value':
        # Received an event dictionary, extract the selected value
        selected_warehouse = change['new']
    else:
        # Invalid change, do nothing
        return

    print(f"Selected Warehouse: {selected_warehouse}")
    getWarehousePartsAPI(selected_warehouse)

In [216]:
# Trigger on_selection_change with the default value
on_selection_change(selection.value)

Selected Warehouse: 2


In [217]:
# Attach the function to the widget's value change event
selection.observe(on_selection_change)
display(selection)
display(output_area)

Dropdown(description='wid', options=(2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 15, 9, 1), value=2)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '    Color Material         Name  id   …